In [1]:
from tensorflow import keras
import numpy as np

f_mnist = keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = f_mnist.load_data()

x_train_full = np.reshape(x_train_full, (x_train_full.shape[0], x_train_full.shape[1] * x_train_full.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2]))

print(f"Training dataset: {x_train_full.shape}, {x_train_full.dtype}")
print(f"Testing dataset: {x_test.shape}, {x_test.dtype}")

x_train = x_train_full / 255.0
from sklearn.preprocessing import StandardScaler
y_train = y_train_full

2024-02-28 09:44:14.620374: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 09:44:14.622716: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 09:44:14.651531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 09:44:14.651558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 09:44:14.652302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Training dataset: (60000, 784), uint8
Testing dataset: (10000, 784), uint8


In [2]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [3]:
class_names = [
    'T-shirt',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

class_names[y_train[0]]

'Ankle boot'

In [4]:
from sklearn.decomposition import PCA


pca = PCA(n_components=392)
x_train_red = pca.fit_transform(x_train_scaled)
x_test_red = pca.transform(x_test_scaled)

print(f"Training dataset: {x_train_red.shape}, {x_train_red.dtype}")
print(f"Testing dataset: {x_test_red.shape}, {x_test_red.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (10000, 392), float64


In [5]:
from sklearn.random_projection import GaussianRandomProjection


rp = GaussianRandomProjection(n_components=392)
x_train_rp = rp.fit_transform(x_train_scaled)
x_test_rp = rp.transform(x_test_scaled)

print(f"Training dataset: {x_train_rp.shape}, {x_train_rp.dtype}")
print(f"Testing dataset: {x_train_rp.shape}, {x_train_rp.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (60000, 392), float64


In [6]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=392)
x_train_tsne = rp.fit_transform(x_train_scaled)
x_test_tsne = rp.transform(x_test_scaled)

print(f"Training dataset: {x_train_tsne.shape}, {x_train_tsne.dtype}")
print(f"Testing dataset: {x_train_tsne.shape}, {x_train_tsne.dtype}")

Training dataset: (60000, 392), float64
Testing dataset: (60000, 392), float64


In [7]:
from sklearn.tree import DecisionTreeClassifier


classifier_tree = DecisionTreeClassifier()

In [8]:
from sklearn.neighbors import KNeighborsClassifier


classifier_kmeans = KNeighborsClassifier()

In [9]:
from sklearn.svm import SVC


classifier_svc = SVC()

In [10]:
from sklearn.linear_model import LogisticRegression


classifier_regression = LogisticRegression()

In [11]:
classifying_models = {
    'Decision Tree': classifier_tree,
    'K-Means': classifier_kmeans,
    'C-Support Vector': classifier_svc,
    'Logistic Regression (aka logit, MaxEnt)': classifier_regression
}

classifying_analyze = {
    'Original dataset': [],
    'PCA': [],
    'Random Projection': [],
    'TSNE': []
}

classifying_analyze_time = {
    'Original dataset': [],
    'PCA': [],
    'Random Projection': [],
    'TSNE': []
}

In [12]:
import time


for model_name, model in classifying_models.items():
    start = time.time()
    model.fit(x_test, y_test)
    end = time.time()
    
    classifying_analyze_time['Original dataset'].append(end-start)
    classifying_analyze['Original dataset'].append(model.score(x_test, y_test))

    start = time.time()
    model.fit(x_test_red, y_test)
    end = time.time()
    
    classifying_analyze_time['PCA'].append(end-start)
    classifying_analyze['PCA'].append(model.score(x_test_red, y_test))

    start = time.time()
    model.fit(x_test_rp, y_test)
    end = time.time()
    
    classifying_analyze_time['Random Projection'].append(end-start)
    classifying_analyze['Random Projection'].append(model.score(x_test_rp, y_test))

    start = time.time()
    model.fit(x_test_tsne, y_test)
    end = time.time()
    
    classifying_analyze_time['TSNE'].append(end-start)
    classifying_analyze['TSNE'].append(model.score(x_test_tsne, y_test))

/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/suphomie/uni/homeworks-labs/machine-learning-tools/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opti

In [13]:
classifying_analyze, classifying_analyze_time

({'Original dataset': [1.0, 0.8675, 0.891, 0.8774],
  'PCA': [1.0, 0.8757, 0.8682, 0.8713],
  'Random Projection': [1.0, 0.868, 0.8677, 0.8721],
  'TSNE': [1.0, 0.8665, 0.8682, 0.867]},
 {'Original dataset': [5.141215562820435,
   0.0018579959869384766,
   18.752952337265015,
   3.9043595790863037],
  'PCA': [7.2454540729522705,
   0.004645586013793945,
   8.524399995803833,
   3.1005330085754395],
  'Random Projection': [8.402970314025879,
   0.00675654411315918,
   8.336491107940674,
   2.993973970413208],
  'TSNE': [12.896084785461426,
   0.007767438888549805,
   8.29047679901123,
   2.7818987369537354]})

In [14]:
classifying_analyze['Model'] = ['Decision Tree', 'K-Means', 'C-Support Vector', 'Logistic Regression (aka logit, MaxEnt)']
classifying_analyze_time['Model'] = ['Decision Tree', 'K-Means', 'C-Support Vector', 'Logistic Regression (aka logit, MaxEnt)']
classifying_analyze
# , classifying_analyze_time

{'Original dataset': [1.0, 0.8675, 0.891, 0.8774],
 'PCA': [1.0, 0.8757, 0.8682, 0.8713],
 'Random Projection': [1.0, 0.868, 0.8677, 0.8721],
 'TSNE': [1.0, 0.8665, 0.8682, 0.867],
 'Model': ['Decision Tree',
  'K-Means',
  'C-Support Vector',
  'Logistic Regression (aka logit, MaxEnt)']}

In [15]:
import pandas as pd


data = classifying_analyze
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original dataset,PCA,Random Projection,TSNE
Model,,,,
Decision Tree,1.0000,1.0000,1.0000,1.0000
K-Means,0.8675,0.8757,0.8680,0.8665
C-Support Vector,0.8910,0.8682,0.8677,0.8682
"Logistic Regression (aka logit, MaxEnt)",0.8774,0.8713,0.8721,0.8670


In [16]:
data = classifying_analyze_time
df = pd.DataFrame.from_dict(data)

df.set_index('Model')

,Original dataset,PCA,Random Projection,TSNE
Model,,,,
Decision Tree,5.141216,7.245454,8.402970,12.896085
K-Means,0.001858,0.004646,0.006757,0.007767
C-Support Vector,18.752952,8.524400,8.336491,8.290477
"Logistic Regression (aka logit, MaxEnt)",3.904360,3.100533,2.993974,2.781899
